In [39]:
import numpy as np
import pandas as pd
import xarray as xr
from rasterio.enums import Resampling

from pathlib import Path

import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec

In [40]:
from scipy.stats import pearsonr
from sklearn.metrics import mean_squared_error, mean_absolute_error

def bias(x, y): return np.mean(x - y)

def get_stats(x, y):
    if type(x) == xr.DataArray: x = x.values.ravel()
    if type(y) == xr.DataArray: y = y.values.ravel()
    if type(x) == list: x = np.array(x)
    if type(y) == list: y = np.array(y)
    idx = (~np.isnan(x)) & (~np.isnan(y))
    x, y = x[idx], y[idx]
    r, p = pearsonr(x, y)
    b = bias(x, y)
    mae = mean_absolute_error(x, y)
    rmse = mean_squared_error(x, y, squared = False)
    return r, b, mae, rmse

In [47]:
in_dir = Path('~/scratch/spicy/SnowEx-Data/').expanduser().resolve()
in_dir = Path('~/spicy-snow/SnowEx-Data-Hans/').expanduser().resolve()

out_dir = Path('~/scratch/spicy/SnowEx-Hans-s1').expanduser()

dss = {fp.stem: xr.open_dataset(fp) for fp in in_dir.glob('*.nc')}

data_dir = Path('~/scratch/West_US_Canada').expanduser()
assert data_dir.exists()
fps = list(data_dir.glob('*.nc'))

In [49]:
for stem, old_ds in dss.items():
    new_dss = []
    for fp in fps:
        dt = pd.to_datetime(fp.stem.split('_')[1])
        if dt > pd.to_datetime('2020-08-01') and dt < pd.to_datetime('2021-04-01'):
            ds = xr.open_dataset(fp)['snd']
            ds = ds.expand_dims(time = [dt])
            ds = ds.sel(lat = slice(old_ds.y.max(), old_ds.y.min()), lon = slice(old_ds.x.min(), old_ds.x.max()))
            new_dss.append(ds)

    full = xr.concat(new_dss, dim = 'time')
    full = full.dropna(dim ='time', how ='all').sortby('time')
    full = full.rename({'lat':'y'})
    full = full.rename({'lon':'x'})

    full.to_netcdf(out_dir.joinpath(stem+'.nc'))

: 

In [5]:
b_21 = xr.open_dataset('~/spicy-snow/SnowEx-Data-Hans/Banner_2021-03-15.nc')['snow_depth'].transpose('time', 'y', 'x').rio.write_crs('EPSG:4326').drop('relative_orbit')
b_20 = xr.open_dataset('~/spicy-snow/SnowEx-Data-Hans/Banner_2020-02-18.nc')['snow_depth'].transpose('time', 'y', 'x').rio.write_crs('EPSG:4326').drop('relative_orbit')
b_20 = b_20.rio.reproject_match(b_21)
b = xr.concat([b_20, b_21], dim ='time')

banner_hans = xr.open_dataarray(data_dir.joinpath('combo.nc')).dropna(dim ='time', how ='all').sortby('time')
banner_hans = banner_hans.rename({'lat':'y'})
banner_hans = banner_hans.rename({'lon':'x'})

banner_hans= banner_hans.rio.write_crs('EPSG:4326')
banner_hans = banner_hans.rio.write_nodata(np.nan)
b = b.rio.write_crs('EPSG:4326')
b = b.rio.write_nodata(np.nan)
b= b.interpolate_na('x', limit = 10)
b= b.sortby('y').interpolate_na('y', limit=10)
b = b.interp_like(banner_hans, method = 'linear') # .interpolate_na('x', method = 'nearest').sortby('y').interpolate_na('y', method = 'nearest')

mg = xr.merge([b, banner_hans])